In [1]:
import json
import pandas as pd
import pandas as pd
import pandas.api.types
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

class_names = {0: 'aeroplane',
               1: 'bicycle',
               2: 'bird',
               3: 'boat',
               4: 'bottle',
               5: 'bus',
               6: 'car',
               7: 'cat',
               8: 'chair',
               9: 'cow',
               10: 'diningtable',
               11: 'dog',
               12: 'horse',
               13: 'motorbike',
               14: 'person',
               15: 'pottedplant',
               16: 'sheep',
               17: 'sofa',
               18: 'train',
               19: 'tvmonitor'}

# Function to convert center-based bounding boxes to COCO format (x_min, y_min, width, height)
def convert_bbox(x_center, y_center, width, height, img_width, img_height):
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    return [x_min, y_min, width * img_width, height * img_height]

# Function to convert ground truth DataFrame to COCO ground truth format
def df_to_coco_ground_truth(df):
    coco_output = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    image_ids = []
    annotations = []
    categories = []
    
    annotation_id = 1
    for _, row in df.iterrows():
        image_id = row['image_id']
        class_id = int(row['class_id'])
        x_center = float(row['x_center'])
        y_center = float(row['y_center'])
        width = float(row['width'])
        height = float(row['height'])
        img_width = float(row['image_width'])   # Get the specific width for this image
        img_height = float(row['image_height']) # Get the specific height for this image
        
        # Add image information (assuming unique image_ids)
        if image_id not in image_ids:
            image_ids.append(image_id)
            coco_output['images'].append({
                "id": int(image_id.split('.')[0]),
                "file_name": image_id,  # Adjust the extension if necessary
                "width": img_width,
                "height": img_height
            })
        
        # Add category information if class_id not seen before
        if class_id not in categories:
            categories.append(class_id)
            coco_output['categories'].append({
                "id": class_id,
                "name": class_names[class_id]  # Assign a default name like "class_x"
            })
        
        # Convert bbox from center format to COCO format
        bbox = convert_bbox(x_center, y_center, width, height, img_width, img_height)
        
        # Add annotation information
        annotations.append({
            "id": annotation_id,
            "image_id": int(image_id.split('.')[0]),
            "category_id": class_id,
            "bbox": bbox,
            "area": bbox[2] * bbox[3],  # area = width * height
            "iscrowd": 0  # Assuming no crowd annotations
        })
        annotation_id += 1
    
    coco_output['annotations'] = annotations
    return coco_output

# Function to convert predictions DataFrame to COCO predictions format
def df_to_coco_predictions(df):
    predictions = []

    for _, row in df.iterrows():
        image_id = row['image_id']
        class_id = int(row['class_id'])
        confidence = float(row['confidence'])
        x_center = float(row['x_center'])
        y_center = float(row['y_center'])
        width = float(row['width'])
        height = float(row['height'])
        img_width = float(row['image_width'])   # Get the specific width for this image
        img_height = float(row['image_height']) # Get the specific height for this image
        
        # Convert bbox from center format to COCO format
        bbox = convert_bbox(x_center, y_center, width, height, img_width, img_height)
        
        # Add prediction entry
        predictions.append({
            "image_id": int(image_id.split('.')[0]),
            "category_id": class_id,
            "bbox": bbox,
            "score": confidence
        })
    
    return predictions

# Function to write the output COCO JSON files
def write_coco_json(output_data, output_path):
    with open(output_path, 'w') as json_file:
        json.dump(output_data, json_file, indent=4)

In [2]:
class ParticipantVisibleError(Exception):
    pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculates the mean Average Precision (mAP) at IoU threshold 0.50 (mAP@50) for object detection predictions.

    This function evaluates the quality of predicted bounding boxes against the ground truth using the COCO evaluation metric.
    It converts both the ground truth and predictions from the provided DataFrames to COCO format, then uses the COCO evaluation 
    toolkit to compute the mAP@50.

    Args:
        solution (pd.DataFrame): Ground truth DataFrame containing the bounding boxes and labels.
        submission (pd.DataFrame): Predicted bounding boxes and labels in a DataFrame format.
        row_id_column_name (str): The name of the column in both DataFrames that contains the unique image identifiers.

    Returns:
        float: The mAP@IoU=0.50 score, which represents the quality of the predictions with respect to the ground truth.
    """
    
    ground_truth_json = 'groundtruth_coco.json'
    predicted_json = 'predictions_coco.json'
    
    # Convert ground truth DataFrame to COCO format
    coco_gt_data = df_to_coco_ground_truth(solution)
    write_coco_json(coco_gt_data, ground_truth_json)
    
    # Convert predictions DataFrame to COCO format
    coco_predictions_data = df_to_coco_predictions(submission)
    write_coco_json(coco_predictions_data, predicted_json)
    
    # Load Ground Truth Annotations
    coco_gt = COCO(ground_truth_json)
    
    # Load Predicted Annotations
    coco_dt = coco_gt.loadRes(predicted_json)
    
    # Initialize COCOeval
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    
    # Set IoU threshold to 0.5 for mAP50
    # coco_eval.params.iouThrs = [0.5]
    # print(coco_eval.params.imgIds)
    
    # Evaluate the detections
    coco_eval.evaluate()
    coco_eval.accumulate()
    # coco_eval.summarize()
    
    # Get the specific mAP@IoU=0.50 score
    mAP_50 = coco_eval.stats[0]  # This corresponds to the mAP@0.50
    return mAP50